# 코랩에서 대화형 챗봇 맛보기 & 허깅페이스 배포
- Google AI Studio 모델과 API 사용 확인: https://ai.google.dev/gemini-api/docs?hl=ko
- 대용량 모델이면서 무료 쿼터가 많은 'gemma-3-27b-it' 사용(30 RPM, 15000 RPD). gemini-2.5-flash-lite도 무료 쿼터 많은 편(15 RPM, 1000 RPD)

### Google AI Studio에서 Gemini API 키 만들기

1. **Google AI Studio 접속**

   * 주소: [https://aistudio.google.com/app/apikey](https://aistudio.google.com/app/apikey)

2. **Google 계정으로 로그인**

3. **API Key 생성**

   * “Create API Key” 버튼 클릭
   * 프로젝트 선택 또는 새 프로젝트 생성
   * 발급된 API Key 보임(한 줄짜리 긴 문자열)

4. **API Key 복사**

   * 이 키는 절대 외부에 공개하면 안 됨.
   * Hugging Face Spaces에 배포할 때는 `secrets` 기능에 넣으면 안전

## 코랩에서 대화형 챗봇 맛보기

In [ ]:
!pip install streamlit google-generativeai

In [ ]:
# Google AI Studio API 키 설정 (코랩에서 보안비밀에 입력)

from google.colab import userdata
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [ ]:
import google.generativeai as genai

# API 키 설정
genai.configure(api_key=GOOGLE_API_KEY)

# 대화 기록 저장
messages = []
system_prompt = "당신은 친절한 AI 어시스턴트입니다."

def chat():
    model = genai.GenerativeModel('gemma-3-27b-it')

    print("=== Google AI Studio 챗봇 ===")
    print("종료하려면 'quit' 입력")
    print("시스템 프롬프트 변경: '/system 새로운 프롬프트'")
    print("-" * 40)

    while True:
        user_input = input("\n사용자: ")

        if user_input.lower() == 'quit':
            break

        if user_input.startswith('/system'):
            global system_prompt
            system_prompt = user_input[7:].strip()
            print(f"시스템 프롬프트 변경됨: {system_prompt}")
            continue

        # 대화 컨텍스트 구성
        context = f"시스템: {system_prompt}\n"
        for msg in messages[-10:]:  # 최근 10개 대화만 유지
            context += f"사용자: {msg['user']}\nAI: {msg['ai']}\n"
        context += f"사용자: {user_input}\nAI:"

        # 응답 생성
        try:
            response = model.generate_content(context)
            ai_response = response.text

            print(f"AI: {ai_response}")

            # 대화 기록에 추가
            messages.append({"user": user_input, "ai": ai_response})

        except Exception as e:
            print(f"오류: {e}")

# 실행
chat()

##  Hugging Face Spaces 배포:
- Streamlit 활용
- 아래 코드를 Hugging Face Spaces에 업로드

### Hugging Face 계정 만들기

1. **회원가입**

   * [https://huggingface.co/join](https://huggingface.co/join)
   * 이메일 / GitHub / Google 계정으로 가입 가능

2. **계정 설정**

   * 프로필 이름(username)을 설정 (Spaces URL에 포함됨)
   * 예: `huggingface.co/yourname`

3. **Access Token 생성 (선택)**

   * Settings → Access Tokens → New Token
   * Spaces에서 private 저장소를 관리하려면 필요 (공개 Space만 만들면 생략 가능)

### Hugging Face Spaces에서 Streamlit 챗봇 배포 준비

1. **New Space 생성**

   * [https://huggingface.co/spaces](https://huggingface.co/spaces) → “New Space” 클릭
   * 이름 입력 (예: `gemma-chatbot`)
   * New Space 클릭 → SDK로 Docker 선택
   * Docker 하위 템플릿 중 Streamlit 선택 → dockerfile 생성
   * Visibility: Public(공개) 또는 Private(비공개) 선택

2. **Secrets 설정**

   * 생성된 Space의 `Settings → secrets` 메뉴로 이동
   * `New secret` 클릭
   * Name: `GOOGLE_API_KEY`
   * Value: Google AI Studio에서 발급받은 API Key 붙여넣기

3. **코드 업로드**

   * 위 코드를 Files > src > **streamlit_app.py**에 덮어씌우기
   * `requirements.txt` 예시:

     ```
     streamlit
     google-generativeai
     ```

4. **빌드 & 실행**

   * Hugging Face가 자동으로 빌드 후 앱을 실행합니다.
   * Embed This Space’ 클릭.
   * URL 예시: `https://huggingface.co/spaces/yourname/gemini-chatbot`

In [ ]:
# streamlit_app.py

import streamlit as st
import google.generativeai as genai
import os
import json
from io import StringIO

# ------------------------------------------------------
# Hugging Face Spaces에서는 환경변수(Secrets)로 API 키를 저장
# Settings → secrets → New secret: GEMINI_API_KEY 등록
# ------------------------------------------------------
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# 초기 설정
if "messages" not in st.session_state:
    st.session_state.messages = []

if "system_prompt" not in st.session_state:
    st.session_state.system_prompt = "당신은 친절한 AI 어시스턴트입니다."

model = genai.GenerativeModel("gemma-3-27b-it")

# ------------------------------------------------------
# Streamlit UI
# ------------------------------------------------------
st.set_page_config(page_title="Gemini 챗봇", page_icon="🤖", layout="wide")
st.title("🤖 Google Gemini 대화형 챗봇")
st.caption("Hugging Face Spaces + Streamlit + Google Generative AI")

# 사이드바: 시스템 프롬프트 수정 & 로그 다운로드
with st.sidebar:
    st.subheader("⚙️ 설정")
    new_system_prompt = st.text_area("시스템 프롬프트", st.session_state.system_prompt, height=100)
    if st.button("변경 적용"):
        st.session_state.system_prompt = new_system_prompt
        st.success("시스템 프롬프트가 변경되었습니다.")

    st.markdown("---")
    if st.session_state.messages:
        # JSON 변환
        json_data = json.dumps(st.session_state.messages, ensure_ascii=False, indent=2)
        st.download_button(
            label="💾 대화 로그 JSON 다운로드",
            data=json_data,
            file_name="chat_log.json",
            mime="application/json"
        )

# 대화 출력
for msg in st.session_state.messages:
    with st.chat_message("user"):
        st.markdown(msg["user"])
    with st.chat_message("assistant"):
        st.markdown(msg["ai"])

# 사용자 입력
if prompt := st.chat_input("메시지를 입력하세요. 종료하려면 'quit' 입력"):
    if prompt.lower() == "quit":
        st.warning("대화를 종료합니다. 새로고침하면 다시 시작할 수 있습니다.")
    else:
        # 대화 컨텍스트 구성
        context = f"시스템: {st.session_state.system_prompt}\n"
        for msg in st.session_state.messages[-10:]:  # 최근 10개만 유지
            context += f"사용자: {msg['user']}\nAI: {msg['ai']}\n"
        context += f"사용자: {prompt}\nAI:"

        with st.chat_message("user"):
            st.markdown(prompt)

        try:
            response = model.generate_content(context)
            ai_response = response.text
        except Exception as e:
            ai_response = f"⚠️ 오류 발생: {e}"

        with st.chat_message("assistant"):
            st.markdown(ai_response)

        # 대화 기록 저장
        st.session_state.messages.append({"user": prompt, "ai": ai_response})